# Chapter 1: Set up data

Before executing the following chapters, one dataset needs to be downloaded and some data needs to be modified slightly. To do this, please follow the steps below.

## 1. Get gene dependencies dataset from the Achilles project website

Watch [this quick video](https://www.youtube.com/watch?v=wj0cJC9-XYw&feature=youtu.be) or follow these steps:

1. Go [here](https://portals.broadinstitute.org/achilles/users/sign_up) to make an account with Project Achilles.
2. Confirm your email with the confirmation link in an email you'll receive from Project Achilles.
3. Go [here](https://portals.broadinstitute.org/achilles/datasets/15/download) and click the "ExpandedGeneZSolsCleaned.csv" link to download the gene dependencies dataset
4. Move the downloaded "ExpandedGeneZSolsCleaned.csv" file to the onco-gps-paper-analysis/data directory


## 2. Prepare data

Run all the cells below.

### Set up notebook and import [CCAL](https://github.com/KwatME/ccal)

In [9]:
#from notebook_environment import *
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import gzip
import itertools
import os
import pickle
import re
import sys
from collections import OrderedDict
from pprint import pprint
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import ccal

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Unzip and prepare datasets

In [12]:
# Unzip data
#ccal.unzip('../data/CTRPv2.2_2015_pub_CancerDisc_5_1210.zip')
ccal.unzip('../data/gene_set__gene_set_x_ccle_cellline.gct.zip')
#ccal.unzip('../data/CCLE_MUT_EXPR_RPPA_OncoGPS.zip')
#for fn in [
        #'mutation__gene_x_ccle_cellline.gct',
        #'rpkm__gene_x_ccle_cellline.gct',
        #'rppa__protein_x_ccle_cellline.gct',
        #'annotation__feature_x_ccle_cellline.gct',
#]:
    #shutil.move('../data/CCLE_datasets/{0}'.format(fn), '../data/{0}'.format(fn))

#Rename Achilles RNAi dataset
#df = pd.read_csv('../data/ExpandedGeneZSolsCleaned.csv', index_col=0)
#ccal.write_gct(df, '../data/achilles__gene_x_ccle_cellline.gct')

# Read compound data
auc = pd.read_table('../data/v22.data.auc_sensitivities.txt')
print(auc.shape)

cpd = pd.read_table('../data/v22.meta.per_compound.txt', index_col=0)
print(cpd.shape)

ccl = pd.read_table('../data/v22.meta.per_cell_line.txt', index_col=0)
print(ccl.shape)

# Make dict for faster ID-to-name look up
cpd_d = cpd['cpd_name'].to_dict()
ccl_d = ccl['ccl_name'].to_dict()

# Make empty compound-x-cellline matrix
compound_x_cellline = pd.DataFrame(
    index=sorted(set(cpd['cpd_name'])), columns=sorted(set(ccl['ccl_name'])))
print(compound_x_cellline.shape)

# Populate compound-x-cellline matrix
for i, (i_cpd, i_ccl, a) in auc.iterrows():

    # Get compound name
    cpd_n = cpd_d[i_cpd]

    # Get cellline name
    ccl_n = ccl_d[i_ccl]

    # Get current AUC
    a_ = compound_x_cellline.loc[cpd_n, ccl_n]

    # If the current AUC is not set, set with this AUC
    if pd.isnull(a_):
        compound_x_cellline.loc[cpd_n, ccl_n] = a

    # If this AUC is smaller than the current AUC, set with this AUC
    elif a < a_:

        print('Updating AUC of compound {} on cellline {}: {:.3f} ==> {:.3f}'.
              format(cpd_n, ccl_n, a_, a))

        compound_x_cellline.loc[cpd_n, ccl_n] = a

# Update cellline names to match CCLE cellline names
columns = list(compound_x_cellline.columns)

# Read CCLE cellline annotations
a = pd.read_table('../data/CCLE_sample_info_file_2012-10-18.txt', index_col=0)

# Get CCLE cellline names
for i, ccl_n in enumerate(compound_x_cellline.columns):

    matches = []

    for ccle_n in a.index:
        if ccl_n.lower() == ccle_n.lower().split('_')[0]:
            matches.append(ccle_n)

    if 0 == len(matches):
        print('0 match: {}; matching substring ...'.format(ccl_n))

        for ccle_n in a.index:

            if ccl_n.lower() in ccle_n.lower():

                print('\t{} ==> {}.'.format(ccl_n, ccle_n))
                matches.append(ccle_n)

    if 1 == len(matches):

        print('{} ==> {}.'.format(ccl_n, matches[0]))
        columns[i] = matches[0]

    else:
        print('1 < matches: {} ==> {}'.format(ccl_n, matches))

# Update with CCLE cellline names
compound_x_cellline.columns = columns

# Write .gct file
ccal.write_gct(compound_x_cellline,
               '../data/ctd2__compound_x_ccle_cellline.gct')

compound_x_cellline

FileNotFoundError: File b'../data/ExpandedGeneZSolsCleaned.csv' does not exist

### Check that all datasets exist

If you get an error running the cell below, get the dataset the error says you're missing, and run it again.

In [ ]:
for fn in [
        'gene_x_kras_isogenic_and_imortalized_celllines.gct',
        'mutation__gene_x_ccle_cellline.gct',
        'rpkm__gene_x_ccle_cellline.gct',
        'gene_set__gene_set_x_ccle_cellline.gct',
        'regulator__gene_set_x_ccle_cellline.gct',
        'rppa__protein_x_ccle_cellline.gct',
        'achilles__gene_x_ccle_cellline.gct',
        'ctd2__compound_x_ccle_cellline.gct',
        'annotation__feature_x_ccle_cellline.gct',
]:
    assert fn in os.listdir('../data'), 'Missing {}!'.format(fn)

### Make the CCLE data object

In [ ]:
# Make the CCLE data object used in coming chapters.

ccle = {
    'Mutation': {
        'df': ccal.read_gct('../data/mutation__gene_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'binary'
    },
    'Gene Expression': {
        'df': ccal.read_gct('../data/rpkm__gene_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Gene Set': {
        'df': ccal.read_gct('../data/gene_set__gene_set_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Regulator Gene Set': {
        'df': ccal.read_gct('../data/regulator__gene_set_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Protein Expression': {
        'df': ccal.read_gct('../data/rppa__protein_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Gene Dependency (Achilles)': {
        'df': ccal.read_gct('../data/achilles__gene_x_ccle_cellline.gct'),
        'emphasis': 'low',
        'data_type': 'continuous'
    },
    'Drug Sensitivity (CTD^2)': {
        'df': ccal.read_gct('../data/ctd2__compound_x_ccle_cellline.gct'),
        'emphasis': 'low',
        'data_type': 'continuous'
    },
    'Primary Site': {
        'df':
        ccal.make_membership_df_from_categorical_series(
            ccal.read_gct('../data/annotation__feature_x_ccle_cellline.gct')
            .loc['Site Primary']),
        'emphasis':
        'high',
        'data_type':
        'binary'
    }
}

with gzip.open('../data/ccle.pickle.gz', 'wb') as f:

    pickle.dump(ccle, f)

### [Next chapter(2)](2 Generate oncogenic-activation signature.ipynb)